In [7]:
import torch
torch.cuda.empty_cache()
torch.cuda.empty_cache()
torch.cuda.empty_cache()
torch.cuda.empty_cache()
torch.cuda.empty_cache()

In [8]:
%%capture
!pip install OmegaConf
!pip install diffusers
!pip install ftfy
!pip install accelerate==0.16.0
!pip install tensorflow-gpu
!pip install pytorch_lightning
!pip install bitsandbytes

In [9]:
!pip install nvidia-pyindex
!pip install --upgrade nvidia-tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-pyindex: filename=nvidia_pyindex-1.0.9-py3-none-any.whl size=8416 sha256=e4906c5f7685a2e49c66c1e194fdb18bd8f2680ac3d321c4755afe733a625966
  Stored in directory: /root/.cache/pip/wheels/f1/a1/a1/6cc45cc1ae6b1876f12ef399c0d0d6e18809e9ced611c7c2a7
Successfully built nvidia-pyindex
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1-py2.py3-none-any.whl size=16985 sha256=b77c379e4fbe916487c4889495b3611fb7ec90db0b93cf3bcef514d97b8c6f78
  Stored in directory: /tmp/pip-ephem-wheel-cache-3h3dlxxh/wheels/da/66/51/3c71ee9327b3e7a7c1d11a780ad7cbd369a4ba0a7ec1edd0c4
Successfully built tensorrt


In [10]:
!pip install --upgrade transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [26]:
!git clone https://github.com/CrazyBoyM/dreambooth-for-diffusion.git

Cloning into 'dreambooth-for-diffusion'...
remote: Enumerating objects: 105, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 105 (delta 22), reused 20 (delta 20), pack-reused 78
Receiving objects: 100% (105/105), 84.47 KiB | 10.56 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [27]:
%cd /kaggle/working/dreambooth-for-diffusion

/kaggle/working/dreambooth-for-diffusion


In [29]:
!wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4-full-ema.ckpt -P ./ckpt_models

--2023-05-26 17:29:59--  https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt
Resolving huggingface.co (huggingface.co)... 18.172.170.70, 18.172.170.14, 18.172.170.44, ...
Connecting to huggingface.co (huggingface.co)|18.172.170.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b4/71/b47143176d3790e957485b59cc13cf072a4b2cbe3340d1b8fa86f53d7197236f/ad2a33c361c1f593c4a1fb32ea81afce2b5bb7d1983c6b94793a26a3b54b08a0?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27v2-1_768-ema-pruned.ckpt%3B+filename%3D%22v2-1_768-ema-pruned.ckpt%22%3B&Expires=1685376762&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2I0LzcxL2I0NzE0MzE3NmQzNzkwZTk1NzQ4NWI1OWNjMTNjZjA3MmE0YjJjYmUzMzQwZDFiOGZhODZmNTNkNzE5NzIzNmYvYWQyYTMzYzM2MWMxZjU5M2M0YTFmYjMyZWE4MWFmY2UyYjViYjdkMTk4M2M2Yjk0NzkzYTI2YTNiNTRiMDhhMD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLC

In [31]:
!python tools/ckpt2diffusers.py \
    --checkpoint_path=./ckpt_models/sd-v1-4-full-ema.ckpt \
    --dump_path=./model \
    --original_config_file=./ckpt_models/model.yaml \
    --scheduler_type="ddim"

--2023-05-26 17:31:23--  https://raw.githubusercontent.com/CompVis/stable-diffusion/main/configs/stable-diffusion/v1-inference.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1873 (1.8K) [text/plain]
Saving to: ‘v1-inference.yaml’

v1-inference.yaml   100%[===================>]   1.83K  --.-KB/s    in 0s      

2023-05-26 17:31:23 (18.8 MB/s) - ‘v1-inference.yaml’ saved [1873/1873]

╭───────────────────── Traceback (most recent call last) ──────────────────────╮
│ /kaggle/working/dreambooth-for-diffusion/tools/ckpt2diffusers.py:797 in      │
│ <module>                                                                     │
│                                                                              │
│   794 │   )                                        

In [ ]:
!mkdir datasets/data

In [ ]:
from datasets import load_dataset
from PIL import Image
import os
import io

dataset = load_dataset("logo-wizard/modern-logo-dataset")
output_folder = 'datasets/data'

for i, example in enumerate(dataset["train"]):
    image = example["image"]
    image.save(os.path.join(output_folder, example["text"][:min(len(example["text"]), 200)].replace("/","-")+".png"))

In [ ]:
%%time
!accelerate launch tools/train_dreambooth.py \
  --pretrained_model_name_or_path="./model"  \
  --mixed_precision="fp16" \
  --instance_data_dir="/kaggle/working/dreambooth-for-diffusion/datasets/data" \
  --class_data_dir="/kaggle/working/dreambooth-for-diffusion/datasets/data" \
  --class_prompt="logo" \
  --use_8bit_adam \
  --use_filename_as_label \
  --output_dir="./new_model" \
  --center_crop \
  --resolution=512 \
  --train_batch_size=1 \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 --gradient_checkpointing \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=1000 \
  --save_model_every_n_steps=1000

### Test the model

In [ ]:
from diffusers import StableDiffusionPipeline
import torch
import matplotlib.pyplot as plt
from diffusers import DDIMScheduler

model_path = "./model"  #new_model
prompt = "a logo of a babershop, a man beard, black foreground, white background, minimalism, modern"
prompt = "a logo of a coffeeshop, a red mug, red foreground, white background, minimalism, modern"
prompt = "a logo of a parrot, minimalism, modern"
prompt = "a logo of a cow, minimalism, modern"
prompt = "a logo of a car company, front view of a blue car, minimalism, modern"

pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        scheduler=DDIMScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            clip_sample=False,
            set_alpha_to_one=True,
        ),
        safety_checker=None
    )

pipe = pipe.to("cuda")
images = pipe(prompt, num_inference_steps=50, guidance_scale=15.0, num_images_per_prompt=3).images
for i, image in enumerate(images):
    plt.subplot(1,3,i+1)
    plt.imshow(image)
    image.save(f"test-{i}.png")

In [ ]:
!zip -r logo_model new_model

In [ ]:
#!rm logo_model.zip
#!rm -r new_model